<a href="https://colab.research.google.com/github/ArtyomShabunin/PINNModels/blob/main/experiment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Эксперимент №1
Разработка PINN для моделирования газотурбинной установки

In [ ]:
import torch
import torch.nn as nn
import torch.autograd.functional as F

### 1. CompressorMapNetBeta — обучаемая карта по β и скорости вращения

In [ ]:
class CompressorMapNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 3)  # pi_c, m_dot_rel, eta
        )

    def forward(self, beta, speed_rel):
        x = torch.stack([beta, speed_rel], dim=-1)
        out = self.net(x)
        pi_c = out[..., 0].clamp(min=1.01, max=10.0)
        m_dot_rel = out[..., 1].clamp(min=1e-4)
        eta = out[..., 2].clamp(min=0.1, max=1.0)
        return pi_c, m_dot_rel, eta

### 2. Класс `Compressor`

In [ ]:
class Compressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.map = CompressorMapNet()
        self.cp = 1005.0
        self.R = 287.0
        self.A_ref = nn.Parameter(torch.tensor(1.0))  # может быть обучаемой

    def forward(self, p_in, T_in, p_out, speed_rel, n_iter=10):
        # Целевая степень повышения давления
        pi_c_target = (p_out / p_in).clamp(min=1.01)

        # Инициализация β как параметр, подлежащий оптимизации
        beta = torch.full_like(p_in, 0.5, requires_grad=True)

        # Оптимизация β через градиентный спуск (можно заменить на nn.Parameter + оптимизатор)
        for _ in range(n_iter):
            pi_c_pred, _, _ = self.map(beta, speed_rel)
            loss = (pi_c_pred - pi_c_target).pow(2).mean()
            grad = torch.autograd.grad(loss, beta, create_graph=True)[0]
            beta = (beta - 0.05 * grad).detach().clamp(min=0.01).requires_grad_()

        # Финальные значения по оптимизированному β
        pi_c, m_dot_rel, eta = self.map(beta, speed_rel)

        rho_in = p_in / (self.R * T_in)
        m_dot = m_dot_rel * rho_in * self.A_ref * speed_rel

        T_out = T_in * (1 + (pi_c ** ((1.4 - 1) / 1.4) - 1) / eta)

        delta_h = self.cp * (T_out - T_in)
        omega = speed_rel * 2 * torch.pi * 50
        torque = (m_dot * delta_h) / (omega + 1e-6)

        return {
            'm_dot': m_dot,
            'T_out': T_out,
            'torque': torque,
            'beta': beta.detach(),
            'pi_c': pi_c,
            'eta': eta
        }

In [ ]:
import torch
import torch.nn as nn

# Кастомный слой для ограничения параметра в [min_val, max_val]
class ScaledSigmoid(nn.Module):
    def __init__(self, min_val, max_val):
        super().__init__()
        self.min_val = min_val
        self.max_val = max_val

    def forward(self, x):
        return self.min_val + (self.max_val - self.min_val) * torch.sigmoid(x)

class CombustionChamber(nn.Module):
    def __init__(self, LHV=50e6, gamma=1.4, R=287.0):
        super().__init__()
        self.LHV = LHV
        self.gamma = gamma
        self.R = R

        # Параметры без ограничений
        self.raw_eta_comb = nn.Parameter(torch.tensor([0.0]))
        self.raw_k_loss = nn.Parameter(torch.tensor([0.0]))

        # Слои масштабирования sigmoid → [a, b]
        self.scale_eta = ScaledSigmoid(0.8, 1.0)
        self.scale_k_loss = ScaledSigmoid(0.0, 0.5)

    def forward(self, p_in, p_out, T_in, fuel_flow):
        eta_comb = self.scale_eta(self.raw_eta_comb)
        k_loss = self.scale_k_loss(self.raw_k_loss)

        # Учёт аэродинамического сопротивления
        delta_p = k_loss * p_in.clamp(min=1e-3)
        p_chamber = p_in - delta_p

        # Расход воздуха по уравнению изохорного баланса
        air_flow = (fuel_flow * self.LHV * eta_comb) / (self.R * T_in * (self.gamma / (self.gamma - 1)))

        # Температура газов на выходе
        Q_total = fuel_flow * self.LHV * eta_comb
        cp_gas = self.gamma * self.R / (self.gamma - 1)
        m_total = air_flow + fuel_flow
        T_out = T_in + Q_total / (m_total * cp_gas)

        return {
            'air_flow': air_flow,
            'gas_flow': m_total,
            'T_out': T_out,
            'p_out': p_out,
            'eta_comb': eta_comb,
            'k_loss': k_loss
        }


In [ ]:
class PressureTemperatureBC(nn.Module):
    def __init__(self, p_val, T_val):
        super().__init__()
        self.p = nn.Parameter(torch.tensor(p_val))
        self.T = nn.Parameter(torch.tensor(T_val))

    def forward(self):
        return self.p, self.T


class PressureBC(nn.Module):
    def __init__(self, p_val):
        super().__init__()
        self.p = nn.Parameter(torch.tensor(p_val))

    def forward(self):
        return self.p


class SpeedBC(nn.Module):
    def __init__(self, speed_val):
        super().__init__()
        self.speed_rel = nn.Parameter(torch.tensor(speed_val))

    def forward(self):
        return self.speed_rel

In [ ]:
class VolumeNode(nn.Module):
    def __init__(self, volume=0.1, dt=0.01, R=287.0, cp=1005.0):
        super().__init__()
        self.volume = volume  # м³ — объём узла
        self.dt = dt  # временной шаг
        self.R = R
        self.cp = cp

        # Инициализируем давление и температуру в узле (будут обновляться)
        self.register_buffer('p', torch.tensor(1e5))  # Па
        self.register_buffer('T', torch.tensor(300.0))  # К

    def forward(self, m_in, T_in, m_out):
        """
        m_in: подача воздуха в узел (кг/с)
        T_in: температура входящего воздуха (К)
        m_out: расход из узла (кг/с)

        Возвращает: давление и температура в узле на текущем шаге
        """
        # Текущая масса в узле
        m_curr = (self.p * self.volume) / (self.R * self.T)

        # Массовый и энергетический балансы
        m_new = m_curr + self.dt * (m_in - m_out)
        e_curr = m_curr * self.cp * self.T
        e_in = m_in * self.cp * T_in
        e_out = m_out * self.cp * self.T
        e_new = e_curr + self.dt * (e_in - e_out)

        # Обновляем состояние
        self.T = (e_new / (m_new * self.cp)).clamp(min=100.0, max=3000.0)
        self.p = (m_new * self.R * self.T / self.volume).clamp(min=1e3, max=1e7)

        return self.p, self.T

In [ ]:
import torch
import torch.nn as nn

class GTUModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Граничные условия
        self.bc_inlet = PressureTemperatureBC(1e5, 288.15)  # p, T
        self.bc_comb_out = PressureBC(2e5)                  # p
        self.bc_speed = SpeedBC(1.0)                        # относительная скорость

        # Компоненты
        self.volume_node = VolumeNode(volume=0.1, dt=0.01)
        self.compressor = Compressor()                     # с CompressorMapNetBeta
        self.combustion_chamber = CombustionChamber()      # с тепловым балансом и сопротивлением

    def forward_iteration(self, fuel_flow):
        # Граничные условия
        p_inlet, T_inlet = self.bc_inlet()
        p_comb_out = self.bc_comb_out()
        speed_rel = self.bc_speed()

        # Текущее состояние узла
        p_node, T_node = self.volume_node.p, self.volume_node.T

        # Компрессор
        comp_out = self.compressor(p_inlet, T_inlet, p_node, speed_rel)
        m_air_in = comp_out['m_dot']
        T_air_out = comp_out['T_out']

        # Камера сгорания
        comb_out = self.combustion_chamber(p_node, p_comb_out, T_node, fuel_flow)
        m_air_out = comb_out['air_flow']

        # Обновление состояния узла
        p_node_new, T_node_new = self.volume_node(m_air_in, T_air_out, m_air_out)

        return {
            'compressor': comp_out,
            'combustion_chamber': comb_out,
            'volume_node': {'p': p_node_new, 'T': T_node_new}
        }

    def forward(self, fuel_flow, max_iters=100, tol=1e-3):
        # Копии начального состояния (градиенты не нужны)
        prev_p = self.volume_node.p.detach().clone()
        prev_T = self.volume_node.T.detach().clone()

        for i in range(max_iters - 1):
            # Важно: не используем torch.no_grad()
            _ = self.forward_iteration(fuel_flow)

            dp = (self.volume_node.p - prev_p).abs()
            dT = (self.volume_node.T - prev_T).abs()

            if dp < tol and dT < tol:
                break

            prev_p = self.volume_node.p.detach().clone()
            prev_T = self.volume_node.T.detach().clone()

        # Последняя итерация — с сохранением графа
        return self.forward_iteration(fuel_flow)


In [ ]:
model = GTUModel()

# Топливо — входной параметр, требующий градиента
fuel_flow = torch.tensor([0.5], requires_grad=True)

# Прямой проход
output = model(fuel_flow)

# Пример целевой функции: хотим T_out = 1400 K
target_T = torch.tensor([1400.0])
loss = (output["combustion_chamber"]["T_out"] - target_T).pow(2).mean()

# Обратный проход
loss.backward()

# Проверим градиенты
print(fuel_flow.grad)  # dL/dfuel_flow
# print(model.combustion_chamber.raw_eta_comb.grad)  # dL/deta

tensor([7.8001e+08])


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn